In [1]:
import datetime as dt
import inspect
import os
import sys

import ee
from IPython.display import Image
from importlib import reload  # Python 3 only
# from imp import reload      # Python 2/3 via futures modules

from interp import interpolate
# Assume ET models will be installed via pip or available in EE
# For now, add parent folder to path in order to access models in other folders
# This seems super awful, is there a better way to do this in the short term?
root_path = os.path.dirname(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))))
if os.path.join(root_path, 'ndvi-et-ee') not in sys.path:
    sys.path.insert(0, os.path.join(root_path, 'ndvi-et-ee'))
from ndvi_et import ndvi_et

interpolate = reload(interpolate)
ndvi_et = reload(ndvi_et)

ee.Initialize()

et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

## Input parameters

In [2]:
# Date range you want to aggregate ET over
start_date = '2015-08-01'
end_date = '2015-08-31'

# Only keep images with an average cloud cover less than
cloud_cover = 70

# Number of extra days (at start and end) to include in interpolation
interp_days = 32
# Interpolation method - currently only LINEAR is supported
interp_type = 'LINEAR'

study_area = ee.Geometry.Rectangle(-119.25, 38.80, -119.05, 39.15)

In [3]:
# Studay area properties
study_region = study_area.bounds(1, 'EPSG:4326').coordinates().getInfo()
study_crs = 'EPSG:32611'
# study_crs = landsat_img.select(['B2']).projection().crs().getInfo()
# study_transform = landsat_img.select(['B2']).projection().getInfo()['transform']

In [4]:
# Add extra Landsat images at start and end to interpolate between
interp_start_date = (dt.datetime.strptime(start_date, '%Y-%m-%d') - dt.timedelta(days=interp_days)).strftime('%Y-%m-%d')
interp_end_date = (dt.datetime.strptime(end_date, '%Y-%m-%d') + dt.timedelta(days=interp_days)).strftime('%Y-%m-%d')
print(interp_start_date)
print(interp_end_date)

2015-06-30
2015-10-02


## Build the Landsat TOA Collection

In [5]:
# Build the input Landsat TOA collection
# landsat_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA') \
#     .filterDate(interp_start_date, interp_end_date) \
#     .filterBounds(study_area) \
#     .filterMetadata('CLOUD_COVER_LAND', 'less_than', cloud_cover) \
#     .filterMetadata('DATA_TYPE', 'equals', 'L1TP')

# Code for setting up a merged Landsat input collection
landsat_coll = ee.ImageCollection([])
landsat_coll = ee.ImageCollection(landsat_coll.merge(
    ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA') \
        .filterDate(interp_start_date, interp_end_date) \
        .filterBounds(study_area) \
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', cloud_cover) \
        .filterMetadata('DATA_TYPE', 'equals', 'L1TP')))
landsat_coll = ee.ImageCollection(landsat_coll.merge(
    ee.ImageCollection('LANDSAT/LE07/C01/T1_RT_TOA') \
        .filterDate(interp_start_date, interp_end_date) \
        .filterBounds(study_area) \
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', cloud_cover) \
        .filterMetadata('DATA_TYPE', 'equals', 'L1TP')))
# landsat_coll = ee.ImageCollection(landsat_coll.merge(
#     ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
#         .filterDate(interp_start_date, interp_end_date) \
#         .filterBounds(study_area) \
#         .filterMetadata('CLOUD_COVER_LAND', 'less_than', cloud_cover) \
#         .filterMetadata('DATA_TYPE', 'equals', 'L1TP')))
print(landsat_coll.aggregate_histogram('system:index').getInfo())

{'1_2_LC08_042033_20150713': 1, '1_2_LC08_042033_20150729': 1, '1_2_LC08_042033_20150814': 1, '1_2_LC08_042033_20150830': 1, '1_2_LC08_042033_20150915': 1, '1_2_LC08_043033_20150720': 1, '1_2_LC08_043033_20150805': 1, '1_2_LC08_043033_20150821': 1, '1_2_LC08_043033_20150906': 1, '1_2_LC08_043033_20150922': 1, '2_LE07_042033_20150705': 1, '2_LE07_042033_20150721': 1, '2_LE07_042033_20150822': 1, '2_LE07_042033_20150907': 1, '2_LE07_042033_20150923': 1, '2_LE07_043033_20150728': 1, '2_LE07_043033_20150813': 1, '2_LE07_043033_20150829': 1, '2_LE07_043033_20150914': 1}


In [6]:
Image(url=ee.Image(landsat_coll.first()).select([3, 2, 1]) \
    .getThumbURL({'min': 0.0, 'max': 0.3, 'region': study_region}))

## Apply NDVI ET Model

In [7]:
# Compute ETf for each Landsat scene
def compute_et_fraction(image):
    s = ndvi_et.NDVI_ET.fromLandsatTOA(
        toa_image=ee.Image(image), m=1.25, b=0)
    return ee.Image(s.etf())
scene_et_fraction_coll = ee.ImageCollection(
    landsat_coll.map(compute_et_fraction))

# Daily reference ET collection
daily_et_reference_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET') \
    .filterDate(start_date, end_date) \
    .select(['etr'], ['et_reference'])

# Compute composite/mosaic images for each image date
# This will combine Landsat images from the same path into a single image
daily_et_fraction_coll = ee.ImageCollection(interpolate.aggregate_daily(
    image_coll=scene_et_fraction_coll,
    start_date=interp_start_date,
    end_date=interp_end_date))

# Interpolate daily ETf, multiply by daily ETr, and sum to ET
daily_et_actual_coll = ee.ImageCollection(interpolate.interp_et_coll(
    et_reference_coll=daily_et_reference_coll,
    et_fraction_coll=daily_et_fraction_coll,
    interp_days=interp_days,
    interp_type=interp_type))

## Compute total ET over the time period

In [8]:
Image(url=ee.Image(daily_et_actual_coll.sum()) \
    .reproject(crs=study_crs, scale=30) \
    .getThumbURL({'min': 0.0, 'max': 400, 'region': study_region, 'palette': ','.join(et_palette)}))

## Compute total ETr over the time period

In [9]:
Image(url=ee.Image(daily_et_reference_coll.sum()) \
    .reproject(crs=study_crs, scale=30) \
    .getThumbURL({'min': 0, 'max': 400, 'region': study_region, 'palette': ','.join(et_palette)}))

## Compute Mean ETrF over the time period

In [10]:
Image(url=ee.Image(daily_et_actual_coll.sum()).divide(ee.Image(daily_et_reference_coll.sum())) \
    .reproject(crs=study_crs, scale=30) \
    .getThumbURL({'min': 0.0, 'max': 1.2, 'region': study_region, 'palette': ','.join(et_palette)}))

## Compute Image Count over the time period

In [11]:
# End date should be advanced 1 day
Image(url=ee.Image(daily_et_fraction_coll.filterDate(start_date, end_date).count()) \
    .reproject(crs=study_crs, scale=30) \
    .getThumbURL({'min': 0.0, 'max': 8, 'region': study_region}))